## Potential Energy Surface Python Code
<br>
This script allows the user to enter in a diatomic molecule to compute the Potential Energy Surface through of the Hartree-Fock method. 
<br> <br>
The program is limited to only using Hydrogen and Helium atoms due to the Hartree-Fock base code only supporting s-orbitals. 
<br><br>
The potential energy surface graph uses Hartrees as units for the energy, and bohr units for representing the diatomic bond distance. 
<br><br>
The Vibrational Energy Spectra are also computed and overlayed over the potential energy surface.

In [1]:
#import all required packages here
import math
import scipy.integrate as integrate
import scipy.optimize as optimize
from scipy.misc import derivative as ddx
import numpy as np

from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from morsePotential import morsePotential
from notebookImporter import importNotebook
from tqdm import tqdm

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [8]:
HartreeFock = importNotebook("Hartree_Class")

#set up the automated calculations used to generate the surface potential

#User Specified Variables

#Enter the atomic number for atoms 1 and 2
atom1 = 1
atom2 = 1

#Enter the number of electrons for atom 1 and 2
#Note the total number of electrons must be a multiple of 2, 
#otherwise, the computation will round the total number of electrons 
#down to the nearest multiple of 2
atomN1 = 1
atomN2 = 1

#Select a basis, for a list of available basis sets, please see the "basisSets" folder
basisName = "DEF2-SVP"

#Enter Starting and End Bond distance for the diatomic molecule
#Units for Bond Distance in This Program are Bohr Units
startR = 1
endR = 10

#Please enter the distance the bond distance should increase each iteration
#The smaller the number, the more accurate and time-consuming the computation will be
step = 0.01

#Please enter how small the difference in energy between two ground state energy calculations
#must be in order to convergence to occur
convergenceCritera = pow(10, -10)

#Please enter the maximum number of iterations the SCF procedure can take before
#the computation will fail as being unable to converge
maxIterations = 100

#System Defined Variables
E = []
basisSets = []
Xs = []
MOEnergy = []
currentR = startR
delta = (endR - startR) / step

#Compute the the descrite points in terms of bond distance that will be used to create 
#the potential energy surface
R = [ startR + step*ds for ds in range( math.ceil(delta) + 1) ]

#Main While Loop to repeatedly call the Hartree-Fock routine to generate the potential energy surface
for currentR in tqdm(R, position=0, desc="Hartree-Fock Progress"): 

    #define the molecular system
    system = molecule()
    system.addAtom(atom(vector(1,1,2+currentR), atom1, atomN1))
    system.addAtom(atom(vector(1,1,2), atom2, atomN2))

    system.addBasis(basisName)
    
    #Set up the Hartree Procedure
    HF = HartreeFock.HF(system, convergenceCritera, maxIterations)

    #Store Bond distance, associated Energy, and MO Energies
    E.append(HF.SCF())
    MOEnergy.append(HF.MOEnergy)
    
    #update the current intenuclear distance
    currentR += step
        
print("Hartree-Fock Computation Complete")

#For Graphing the Hartree Potential
figure = { 
    "data": [
        #create the Hartree-Fock generated Potential Energy Surface
        {
            "type":"scatter",
            "x":R,
            "y":E,
            "connectgaps":False,
            "mode":"markers", 
            "name":"Hartree-Fock Computed",
            "marker":{"color":"blue"},
            
        }
    ],
     #Set up the layout of the graph
    "layout":
        {
           "xaxis":{"title":"Bond Distance in Bohr Units Radius"},
           "yaxis":{"title":"Energy in Hartrees"},
            "title":{"text":"Hartree-Fock Computed Potential Energy Surface"}
        },    
}
iplot(figure)

Hartree-Fock Progress: 100%|█████████████████████████████████████████████████████████| 901/901 [01:42<00:00,  8.90it/s]


Hartree-Fock Computation Complete


In [9]:

#create a curve fitted morsePotential of the data
#uses the least squares method to fit the potential via SciPy
#Morse Potential Equation is from "Diatomic Molecules According To 
#The Wave Mechanics. II. Vibrational Levels", by Phillip M. Morse
def morsePotential(r, r0, a, D, c):
    return D * pow( (1-np.exp(-a*(r-r0))), 2) + c

#Optimize the morse potential to the Hartree-Fock computed potential
popt, pcov = optimize.curve_fit(morsePotential, R, E)

#Get important data for graphing purposes
minEIndex = E.index(min(E))
minE = E.copy().pop(minEIndex)
optimalR = R.copy().pop(minEIndex)

#graph the surface potential
dx = .01
figure = { 
    "data": [
        
        #create the Hartree-Fock generated Potential Energy Surface
        {
            "type":"scatter",
            "x":[x*dx for x in range(-100,1000)],
            "y":[morsePotential(x*dx, popt[0], popt[1], popt[2], popt[3]) for x in range(-100,1000)],
            "connectgaps":False,
            "mode":"markers", 
            "name":"Hartree-Fock Computed",
            "marker":{"color":"blue"}
        },
    ],
     "layout":
        {
           "xaxis":{"title":"Bond Distance in Bohr Units Radius"},
           "yaxis":{"title":"Energy in Hartrees"},
            "title":{"text":"Fitted Morse Potential Curve"}
        },    
}
iplot(figure)

print(popt)

[ 1.49597085  1.65108308  0.05332825 -2.90847807]


In [11]:
#basis size to use for the computation 
basisSize = 10

#Correction Value to Set Morse Potential at the Zero Point
correction = morsePotential(popt[0], popt[0], popt[1], popt[2], popt[3])

#compute omega from the variables used to describe the Morse Potential 
#Equation originates from Exact Solutions for Vibrational Levels of the Morse Potential via the 
#asymptotic iteration method 
#by T. Barakat and K. Abodayeh
w = popt[1] * math.sqrt(2*popt[2] / system.u)
uw = w * system.u

#Returns a lambda function of the nth hermite polynomial
def hermite(n, k):
    
    c = pow(-1, k) * math.factorial(n) / ( math.factorial(k) * math.factorial(n - 2*k) )
        
    if(k == 0):
        return lambda r : c * pow(2*math.sqrt(uw)*( r-popt[0] ), n-(2*k))
    else:
        return lambda r : c * pow(2*math.sqrt(uw)*( r-popt[0] ), n-(2*k)) + hermite(n, k-1)(r)

#Constructs the nth hermite polynomial function, and returns the specified lambda function
def buildHermite(n):
    return hermite(n, n // 2)

#returns the normalization constant for the Harmonic Oscillator Function
def C(n):    
    return math.sqrt( math.sqrt(uw) /  (pow(2, n) * math.factorial(n) * np.sqrt(math.pi) )) 

#Creates a new Harmonic Oscillator function
#and returns it as a lambda function
def newHO(n):
    return lambda r : C(n) * buildHermite(n)(r) * np.exp(-uw * pow(r-popt[0],2) / 2)  

#Set up all variables used for the integral computation
basisSet = []
S = np.zeros([basisSize, basisSize])
V = np.zeros([basisSize, basisSize])
T = np.zeros([basisSize, basisSize])
T2 = np.zeros([basisSize, basisSize])

Tints = []
Vints = []

#Build the basis Fucntions
for i in range(basisSize):
    basisSet.append( newHO(i) )
    
#Verify orthonormality of the basis set
for i in range(basisSize):
    for j in tqdm(range(basisSize), desc="Row " + str(i) + " Integral Progress", position=0):
        
        integrand = lambda r : basisSet[i](r) *  (morsePotential(r, popt[0], popt[1], popt[2], popt[3]) - correction) * basisSet[j](r)
        
        V[i, j] += round(integrate.quad( integrand, 0, np.inf)[0], 3)
        integrand2 = lambda r : basisSet[i](r) * (-1/(2*system.u)) * ddx(basisSet[j], r, n=2, dx=pow(10, -4))

        T[i, j] += round(integrate.quad(integrand2, 0, np.inf)[0], 3)
    
        Tints.append(integrand2)
        Vints.append(integrand)

print()
print("Overlap Matrix: ")
print()
print(S)
print()
print("*" * 40)
print()

print("V")
print()
print(V)
print()
print("*"*40)
print()

print("T")
print()
print(T)
print()
print("*" * 40)
print()

H = T + V

print(H)
print()

EHO, EV = np.linalg.eigh(H)

Row 9 Integral Progress: 100%|█████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 32.34it/s]



Overlap Matrix: 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

****************************************

V

[[ 0.004 -0.003  0.007 -0.003  0.001  0.     0.     0.     0.     0.   ]
 [-0.003  0.014 -0.01   0.014 -0.007  0.002 -0.001  0.     0.     0.   ]
 [ 0.007 -0.01   0.026 -0.02   0.022 -0.011  0.004 -0.001  0.     0.   ]
 [-0.003  0.014 -0.02   0.041 -0.032  0.033 -0.017  0.007 -0.002  0.001]
 [ 0.001 -0.007  0.022 -0.032  0.058 -0.047  0.046 -0.025  0.01  -0.003]
 [ 0.     0.002 -0.011  0.033 -0.047  0.079 -0.065  0.061 -0.034  0.014]
 [ 0.    -0.001  0.004 -0.017  0.046 -0.065  0.103 -0.087  0.078 -0.045]
 [ 0.     0.    -0.001  0.007 -0.025  0.061 -0.087  0.13  -0.112  0.099]
 [ 0.     0.     0. 

In [12]:
#prepare data for use with graphing
dx = .01
steps = 15/dx

figure = {
    "data": [
        #create the Hartree-Fock generated Potential Energy Surface
        {
            "type":"scatter",
            "x":R,
            "y":E,
            "connectgaps":False,
            "mode":"markers", 
            "name":"Hartree-Fock Computed",
            "marker":{"color":"blue"}
        },
    
        #Highlight the minimum energy point in red
        {
            "type":"scatter",
            "x":[optimalR],
            "y":[minE],
            "name":"Optimal Bond Distance",
            "marker":{"color":"red"}
        },
        
        #Create and plot the Morse Potential fit
        {
            "type":"scatter",
            "x":[x*dx for x in range(-1, 1000)],
            "y": [morsePotential(r * dx, popt[0], popt[1], popt[2], popt[3]) - correction for r in range(-1, 1000)], #popt[0], popt[1], popt[2], popt[3]) for r in data] ,
            "connectgaps":False,
            "name":"Morse Potential Approximation",
            "marker":{"color":"green"}
        },
    ],
    
    #Set up the layout of the graph
    "layout":
        {
           "xaxis":{"title":"Bond Distance in Bohr Units Radius"},
           "yaxis":{"title":"Energy in Hartrees"},
            "title":{"text":"Hartree-Fock Energy VS Bond Distance"}
        },    
}

xMin = -40
xMax = 40
steps = 500
dx = (xMax - xMin) / steps
        
for i in range(len(EHO)):

    figure["data"].append(
        {
            "type":"scatter",
            "x":[popt[0]],
            "y":[EHO[i].real],
            "connectgaps":False,
            "name":"T" + str(i),
            "marker":{"color":"blue"}
        }
    )    
    
iplot(figure)